In [12]:
import pandas as pd


import numpy as np


import re

In [13]:
def question_type(question):
  question_type = "?"
  #question = row['question']
  if "What is" in question:
    question_type = "quantitative"
  elif "Is there any" in question:
    question_type = "qualitative"

  return question_type

In [14]:
def qualitative_answer_sample(modle_answer):
  if "No Answer" in modle_answer :
    return "No Answer"

  else:
    if "Yes" in modle_answer or 'yes' in modle_answer:
      return "Yes"
    elif "No" in modle_answer or 'no' in modle_answer:
      return "No"


def qualitative_answer_comp(row):
  gold_a = row['gold_Answer']
  modle_a = row['modle_Answer']
  sample_a = qualitative_answer_sample(modle_a)
  if gold_a == gold_a:
    t_answer = gold_a
  else:
    t_answer = "No Answer"

  return t_answer == sample_a

In [16]:
import re
import pandas as pd

def quantitative_answer_sample(answer):
    text = str(answer).replace(",", "")
    
    # Extract numbers and percentages
    matches = re.findall(r'\d+\.\d+%?|\d+%?', text)
    
    filtered_matches = []
    for match in matches:
        if '%' in match:
            value = float(match.strip('%'))
            value /= 100  # Convert percentage to decimal
        else:
            value = float(match)
        filtered_matches.append(value)
    
    return filtered_matches

def quantitative_answer_comp(row):
    g_answer = row['gold_Answer']
    m_answer = row['modle_Answer']
    question = row['question']

    g_answer_s = quantitative_answer_sample(g_answer)
    m_answer_s = quantitative_answer_sample(m_answer)

    no_answers = {"No", "N/A", "Not", "-", "no", "NA", "negligible"}

    m_words = m_answer.split()
    
    if len(g_answer_s) < 1:
        if (len(m_answer_s) == 0) or (0 in m_answer_s) or bool(no_answers & set(m_words)):
            return True
        else:
            return False
    else:
        if len(g_answer_s) > 1:
            num_in_question = quantitative_answer_sample(question)
            g_answer_s = [item for item in g_answer_s if item not in num_in_question]

        if 0 in g_answer_s:
            if (len(m_answer_s) == 0) or (0 in m_answer_s) or bool(no_answers & set(m_words)):
                return True
            else:
                return False
        else:
            if any(abs(m - g) < 1e-6 for m in m_answer_s for g in g_answer_s):
                return True
            else:
                return False

def modle_answer_check_row(row):
    q_type = question_type(row['question'])
    if q_type == 'quantitative':
        return quantitative_answer_comp(row)
    else:
        return qualitative_answer_comp(row)

def modle_answer_check_all(df):
    df['answer_correctness'] = df.apply(lambda row: modle_answer_check_row(row), axis=1)
    return df



In [17]:
def modle_answer_check_row(row):
    q_type = question_type(row['question'])
    if q_type == 'quantitative':
        return quantitative_answer_comp(row)
    else:
        return qualitative_answer_comp(row)

def modle_answer_check_all(df):
    df['answer_correctness'] = df.apply(lambda row: modle_answer_check_row(row), axis=1)
    df['question_type'] = df['question'].apply(question_type)
    
    df['qualitative_correctness'] = df.apply(lambda row: row['answer_correctness'] if row['question_type'] == 'qualitative' else None, axis=1)
    df['quantitative_correctness'] = df.apply(lambda row: row['answer_correctness'] if row['question_type'] == 'quantitative' else None, axis=1)
    
    return df[['answer_correctness', 'qualitative_correctness', 'quantitative_correctness']]




In [18]:
# Define the file paths for the different models
file_paths = {
    'gemma_2_2b': 'gemma-2-2b.xlsx',
    'gemma_2_9b': 'gemma-2-9b.xlsx',
    'gemma_2_27b': 'gemma-2-27b.xlsx',
    'gemma_2b': 'gemma-2b.xlsx',
    'gpt_3.5': 'gpt-3.5.xlsx',
    'gpt_4': 'gpt-4.xlsx',
    'llama_3_8b': 'llama_3_8b.xlsx',
    'llama_2_13b': 'llama-2-13b.xlsx',
    'mistral_7b': 'mistral-7b.xlsx',
    'mistral_nemo_base_2407': 'Mistral-Nemo-Base-2407.xlsx',
    'Qwen_2_1.5b': 'Qwen2_1.5B.xlsx',
    'Qwen_2_7b': 'Qwen2_7B.xlsx',
    'tinyllama': 'tinyllama.xlsx',
    'yi-34b-bnb-4bit': 'yi-34b-bnb-4bit.xlsx'
}

In [19]:
import pandas as pd


def process_file(file_path):
    df = pd.read_excel(file_path)
    
    # Fill missing values or drop rows with missing values if necessary
    df = df.fillna('No Answer')  # or df.dropna()
    
    df = modle_answer_check_all(df)
    return df[['answer_correctness', 'qualitative_correctness', 'quantitative_correctness']]

results = {}
for name, path in file_paths.items():
    try:
        results[name] = process_file(path)
    except Exception as e:
        print(f"Error processing file {path}: {e}")

# Combine all results into a single DataFrame
all_results = pd.concat(results, axis=1)

# Save the combined DataFrame to a new Excel file
all_results.to_excel('combined_results.xlsx')


In [20]:
all_results

c:\Users\12896\AppData\Local\anaconda3\Lib\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
c:\Users\12896\AppData\Local\anaconda3\Lib\site-packages\IPython\core\formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


gemma_2_2b                                                   \
     answer_correctness qualitative_correctness quantitative_correctness   
0                  True                    None                     True   
1                  True                    None                     True   
2                  True                    None                     True   
3                  True                    None                     True   
4                  True                    None                     True   
...                 ...                     ...                      ...   
2034               True                    True                     None   
2035               True                    True                     None   
2036               True                    True                     None   
2037               True                    True                     None   
2038              False                   False                     None   

             gemma_2_9b                                                   \
     answer_correctness qualitative_correctness quantitative_correctness   
0                  True                    None                     True   
1                  True                    None                     True   
2                  True                    None                     True   
3                 False                    None                    False   
4                  True                    None                     True   
...                 ...                     ...                      ...   
2034               True                    True                     None   
2035               True                    True                     None   
2036               True                    True                     None   
2037              False                   False                     None   
2038              False                   False                     None   

            gemma_2_27b                                                   \
     answer_correctness qualitative_correctness quantitative_correctness   
0                  True                    None                     True   
1                  True                    None                     True   
2                  True                    None                     True   
3                  True                    None                     True   
4                  True                    None                     True   
...                 ...                     ...                      ...   
2034               True                    True                     None   
2035               True                    True                     None   
2036               True                    True                     None   
2037               True                    True                     None   
2038               True                    True                     None   

               gemma_2b  ...              Qwen_2_1.5b          Qwen_2_7b  \
     answer_correctness  ... quantitative_correctness answer_correctness   
0                  True  ...                     True               True   
1                 False  ...                     True               True   
2                 False  ...                    False               True   
3                  True  ...                    False               True   
4                  True  ...                    False               True   
...                 ...  ...                      ...                ...   
2034               True  ...                     None               True   
2035               True  ...                     None               True   
2036               True  ...                     None               True   
2037               True  ...                     None               True   
2038               True  ...                     None               True   

                                                         

In [21]:
qualitative_results = all_results.loc[:, (slice(None), 'qualitative_correctness')]
qualitative_results_simplified = qualitative_results.droplevel(1, axis=1)
stats = qualitative_results_simplified.apply(pd.Series.value_counts).fillna(0).astype(int)
stats_T = stats.T
stats_T['Accuracy'] = stats_T[True] / (stats_T[False] + stats_T[True])
stats_T['Accuracy'] = stats_T['Accuracy'].apply(lambda x: f"{x:.1%}")
stats_T.sort_values(by=True, ascending=False)

,False,True,Accuracy
gpt_3.5,57,453,88.8%
Qwen_2_1.5b,58,452,88.6%
Qwen_2_7b,61,449,88.0%
mistral_7b,101,409,80.2%
gpt_4,126,384,75.3%
gemma_2_9b,138,372,72.9%
gemma_2_27b,148,362,71.0%
yi-34b-bnb-4bit,159,351,68.8%
gemma_2b,241,269,52.7%
mistral_nemo_base_2407,298,212,41.6%


In [22]:
quantitative_results = all_results.loc[:, (slice(None), 'quantitative_correctness')]
quantitative_results_simplified = quantitative_results.droplevel(1, axis=1)
stats = quantitative_results_simplified.apply(pd.Series.value_counts).fillna(0).astype(int)
stats_T = stats.T
stats_T['Accuracy'] = stats_T[True] / (stats_T[False] + stats_T[True])
stats_T['Accuracy'] = stats_T['Accuracy'].apply(lambda x: f"{x:.1%}")
stats_T.sort_values(by=True, ascending=False)

,False,True,Accuracy
gpt_3.5,158,1371,89.7%
Qwen_2_7b,206,1323,86.5%
gpt_4,258,1271,83.1%
gemma_2_27b,354,1175,76.8%
gemma_2_2b,394,1135,74.2%
mistral_nemo_base_2407,471,1058,69.2%
gemma_2_9b,520,1009,66.0%
gemma_2b,599,930,60.8%
llama_3_8b,729,800,52.3%
llama_2_13b,861,668,43.7%


In [ ]:
data_path = 'Model_Answer_Comparison_File.xlsx'
comp_data = pd.read_excel(data_path)
df = modle_answer_check_all(comp_data)
df['answer_correctness_all'].value_counts()
df[df['answer_correctness'] == False]['type'].value_counts()#Check the distribution of types of False answers